# Equinox Exploration

CSV's are single-column, first 3 rows are metadata, fourth row is comma-separated column headings

In [82]:
import os
import cudf
import pandas as pd
import cupy as cp

In [83]:
DATA_PATH = '../../data/'
RESULTS_PATH = '../../results/'

Initial CSV Structure

In [84]:
temp = pd.read_csv(DATA_PATH + 'equinox/eqnx_20190119.csv', delimiter='\n')
temp.head(10)

,Expocode: MLCE20190119
0,Ship: Equinox
1,Group: AOML
2,Investigators: Wanninkhof R. ; Pierrot D.
3,"Expocode,YD_UTC,DATE_UTC__ddmmyyyy,TIME_UTC_hh..."
4,"MLCE20190119,19.89876,19012019,21:34:13,25.786..."
5,"MLCE20190119,19.89985,19012019,21:35:47,25.787..."
6,"MLCE20190119,19.90096,19012019,21:37:23,25.789..."
7,"MLCE20190119,19.90206,19012019,21:38:58,25.790..."
8,"MLCE20190119,19.90317,19012019,21:40:34,25.792..."
9,"MLCE20190119,19.90664,19012019,21:45:34,25.799..."


## Cleaning
Removing first three columns containing metadata for ship, sensor, etc

In [85]:
def remove_meta(df):
    cols = df.loc[3].values[0].split(',')
    cols[2] = 'DATE_UTC'
    cols[3] = 'TIME_UTC'
    cols[4] = 'LAT'
    cols[5] = 'LONG'
    return df.loc[4:], cols

temp, cols = remove_meta(temp)

In [86]:
cols

['Expocode',
 'YD_UTC',
 'DATE_UTC',
 'TIME_UTC',
 'LAT',
 'LONG',
 'xCO2_EQU_ppm',
 'xCO2_ATM_ppm',
 'xCO2_ATM_interpolated_ppm',
 'PRES_EQU_hPa',
 'PRES_ATM@SSP_hPa',
 'TEMP_EQU_C',
 'SST_C',
 'SAL_permil',
 'fCO2_SW@SST_uatm',
 'fCO2_ATM_interpolated_uatm',
 'dfCO2_uatm',
 'WOCE_QC_FLAG',
 'QC_SUBFLAG']

Rename columns to remove time-format specification

Formatting functions to split each row by comma and return a pandas series with each value, followed by application of this function to entire dataframe

In [87]:
def split_rows_by_comma(row):
    vals = row.values[0].split(',')
    
    return pd.Series(vals, index=cols)


def split_df_by_comma(df):
    try:
        df = df.to_pandas()
    except Exception:
        pass
    return df.apply(split_rows_by_comma, axis=1).reset_index().drop('index', axis=1)

Read in all files from the Equinox folder and append data to single GDF

In [88]:
# !rm -r ./equinox/.ipynb_checkpoints

df = pd.DataFrame(columns=cols)

for file in os.listdir(DATA_PATH + 'equinox'):
    temp_csv = pd.read_csv(os.path.join(DATA_PATH + 'equinox',file), delimiter='\n')
    temp_df, cols = remove_meta(temp_csv)
    temp_df = split_df_by_comma(temp_df)
    temp_df = cudf.DataFrame.from_pandas(temp_df)
    df = df.append(temp_df)    

In [89]:
float_cols = cols.copy()
non_float_cols = ['Expocode','DATE_UTC', 'TIME_UTC', 'WOCE_QC_FLAG', 'QC_SUBFLAG'] 
[float_cols.remove(i) for i in non_float_cols]
float_cols, cols

for col in float_cols:
    df[col] = df[col].astype('float32')

In [97]:
df['WOCE_QC_FLAG'] = df['WOCE_QC_FLAG'].astype('uint8')
df['DATE_UTC'] = df['DATE_UTC'].astype('uint32')

df = df.drop(['Expocode', 'xCO2_ATM_ppm'], axis=1)

In [98]:
df.head()

,YD_UTC,DATE_UTC,TIME_UTC,LAT,LONG,xCO2_EQU_ppm,xCO2_ATM_interpolated_ppm,PRES_EQU_hPa,PRES_ATM@SSP_hPa,TEMP_EQU_C,SST_C,SAL_permil,fCO2_SW@SST_uatm,fCO2_ATM_interpolated_uatm,dfCO2_uatm,WOCE_QC_FLAG,QC_SUBFLAG
0,19.898760,19012019,21:34:13,25.785999,-80.048798,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2,
1,19.899851,19012019,21:35:47,25.787800,-80.043198,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2,
2,19.900961,19012019,21:37:23,25.789301,-80.036697,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2,
3,19.902060,19012019,21:38:58,25.790701,-80.029503,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2,
4,19.903170,19012019,21:40:34,25.792200,-80.021698,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2,


In [99]:
import tensorflow_data_validation as tfdv

stats = tfdv.generate_statistics_from_dataframe(df)
tfdv.visualize_statistics(stats)

In [100]:
df.to_csv(RESULTS_PATH + 'equinox/equinox-201920.csv', index=False)

# Exploring

In [1]:
import cudf
import cupy as cp
import os
import plotly.graph_objects as go

RESULTS_PATH = '../../results/equinox'

In [2]:
df = cudf.read_csv(os.path.join(RESULTS_PATH,'equinox-201920.csv'))
df.columns

Index(['YD_UTC', 'DATE_UTC', 'TIME_UTC', 'LAT', 'LONG', 'xCO2_EQU_ppm',
       'xCO2_ATM_interpolated_ppm', 'PRES_EQU_hPa', 'PRES_ATM@SSP_hPa',
       'TEMP_EQU_C', 'SST_C', 'SAL_permil', 'fCO2_SW@SST_uatm',
       'fCO2_ATM_interpolated_uatm', 'dfCO2_uatm', 'WOCE_QC_FLAG',
       'QC_SUBFLAG'],
      dtype='object')

In [3]:
df_latlon = df[['DATE_UTC', 'LAT', 'LONG']]
df_latlon.head()

,DATE_UTC,LAT,LONG
0,19012019,25.7860,-80.0488
1,19012019,25.7878,-80.0432
2,19012019,25.7893,-80.0367
3,19012019,25.7907,-80.0295
4,19012019,25.7922,-80.0217


In [4]:
import cuxfilter
import cudf
import tensorflow_data_validation as tfdv

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

demo_red_blue_palette = [ "#3182bd", "#6baed6", "#7b8ed8", "#e26798", "#ff0068" , "#323232" ]


#create cuxfilter dataframe
cux_df = cuxfilter.DataFrame.from_dataframe(df_latlon)
chart0 = cuxfilter.charts.scatter(x='LAT', y='LONG',x_axis_type="mercator", y_axis_type="mercator",  tile_provider='CARTODBPOSITRON', color_palette=demo_red_blue_palette, pixel_shade_type='linear', plot_width=800, plot_height=800)
# chart0.add_tile(tile_provider)

d = cux_df.dashboard([chart0])
chart0.view()

Card(sizing_mode='stretch_both', title='Scatter plot f..., width=800)
    [0] Bokeh(Figure)

In [5]:
df_latlon['DATE_UTC'] = df['DATE_UTC'].astype(str)
df_latlon = df_latlon.sort_values('DATE_UTC').reset_index()

# df_latlon['DATE_UTC'] = df_latlon['DATE_UTC'].to_pandas().apply(lambda x: x.strftime('%Y-%m-%d'))

In [6]:
def fix_date(row):
    full = row['DATE_UTC']
    year = full[-4:]
    month = full[-6:-4]
    day = full[:-6]
    return year + '-' + month + '-' + day

In [7]:
df_latlon.DATE_UTC = df_latlon.to_pandas().apply(fix_date, axis=1)

In [8]:
df_latlon.drop('index',axis=1, inplace=True)

In [9]:
df_latlon.to_pandas().to_csv(os.path.join(RESULTS_PATH, 'df_latlon.csv'), index=False)

In [118]:
stats = tfdv.generate_statistics_from_dataframe(df.to_pandas())
tfdv.visualize_statistics(stats)

In [104]:
df.head()

,YD_UTC,DATE_UTC,TIME_UTC,LAT,LONG,xCO2_EQU_ppm,xCO2_ATM_interpolated_ppm,PRES_EQU_hPa,PRES_ATM@SSP_hPa,TEMP_EQU_C,SST_C,SAL_permil,fCO2_SW@SST_uatm,fCO2_ATM_interpolated_uatm,dfCO2_uatm,WOCE_QC_FLAG,QC_SUBFLAG
19128,1.00024,1774-08-08 22:43:41.128654848,00:00:21,18.9648,-87.4838,409.792,413.29,1015.59,1015.5974,28.02,28.1499,36.2255,396.69,397.78,-1.08,2,<NA>
19129,1.00752,1774-08-08 22:43:41.128654848,00:10:50,18.9872,-87.4663,409.676,413.26,1015.48,1015.6983,28.00,28.1229,36.2615,396.44,397.82,-1.38,2,<NA>
19130,1.00935,1774-08-08 22:43:41.128654848,00:13:28,18.9928,-87.4620,409.812,413.26,1015.63,1015.5980,27.99,28.1196,36.2797,396.75,397.77,-1.02,2,<NA>
19131,1.01142,1774-08-08 22:43:41.128654848,00:16:27,18.9995,-87.4570,410.465,413.25,1015.72,1015.6985,27.99,28.1179,36.2997,397.39,397.81,-0.42,2,<NA>
19132,1.01324,1774-08-08 22:43:41.128654848,00:19:04,19.0053,-87.4527,410.799,413.24,1015.48,1015.7989,28.00,28.1240,36.3160,397.54,397.84,-0.29,2,<NA>
